# In this we look into the performance of log-domain Sinkhorn for different framework namely: numpy, jax and torch.

In [ ]:
import os
# os.environ['XLA_FLAGS'] = (
#     '--xla_gpu_enable_triton_softmax_fusion=true '
#     '--xla_gpu_triton_gemm_any=True '
#     '--xla_gpu_enable_async_collectives=true '
#     '--xla_gpu_enable_latency_hiding_scheduler=true '
#     '--xla_gpu_enable_highest_priority_async_stream=true '
# )

In [ ]:
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)
jax.config.update("jax_platform_name", "cpu")
from __future__ import division
import numpy as np
import time
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
np.random.seed(1234)
import torch
torch.set_default_tensor_type(torch.DoubleTensor)
torch.set_default_dtype(torch.float64)
torch.manual_seed(1234)        
%matplotlib inline 
%load_ext autoreload                                                                                                                                                                                            
%autoreload 

In [ ]:
import computational_OT

In [ ]:
relative_path_to_new_folder = "../Images"
os.makedirs(relative_path_to_new_folder, exist_ok = True)
if not os.path.isdir('../Images/Log_domain_Sinkhorn_varyingframework_images'):
    os.makedirs('../Images/Log_domain_Sinkhorn_varyingframework_images')

## Entropy regularized formulation

The primal entropy regularized formulation of OT is given by:
$$
OT_{\varepsilon}(\alpha,\beta) = min_{\pi \in \mathcal{U}(\alpha,\beta)} \langle C,\pi \rangle +\varepsilon KL(\pi\|\alpha \otimes \beta)\ ,
$$
where
$\ 
KL(\pi\|\alpha \otimes \beta) 
\ $ is the KL-divergence. 

## Entropy regularized dual-formulation
The dual formulation of OT is given by:
$$
OT_{\varepsilon}(\alpha,\beta) = \max_{f\in \mathbb{R}^{n}, g\in\mathbb{R}^{m}} \langle f, \alpha \rangle + \langle g, \beta \rangle - \varepsilon\left(\langle\alpha \otimes \beta, e^{\frac{f}{\varepsilon}}\odot K \odot e^{\frac{g}{\varepsilon}}  \rangle-1\right)\ ,
$$
where
$$
\alpha \in \mathcal{M}_{1}(\mathcal{X}),\ \beta \in \mathcal{M}_{1}(\mathcal{Y}),\ \varepsilon0,\ f\in\mathbb{R}^{n},\ g\in \mathbb{R}^{m}\ .
$$

## Sinkhorn
The optimal coupling $\pi^{*}$ has the following form :
$$
\pi^{*} = diag(\alpha)K diag(\beta)
$$
and we know that $\pi^{*}\mathbb{1}=\alpha$ and $(\pi^{*})^{T}\mathbb{1}=\beta$.
###
Therefore, Sinkhorn updates is given by the following iterative projections
$$
u^{t+1}  \leftarrow \frac{1}{K(v^{t}\odot \beta)}\ , \ 
v^{t+1}  \leftarrow \frac{1}{K^{T}(u^{t+1}\odot \alpha)}\ , 
$$
where 
$K = e^{-\frac{C}{\varepsilon}}\in M_{n\times m}(\mathbb{R}),\ \alpha \in \mathbb{R}^{n},\ \beta \in \mathbb{R}^{m}\ ,\ u\in \mathbb{R}^{n},\ v\in \mathbb{R}^{m}\ and \ (u^{0},v^{0})=(u,v)\ .$



## Log-domain Sinkhorn
Now, the exp-log regularized update of the Sinkhorn algorithm is as follows:
$$
m_{i}(g)\leftarrow \min_{j}(C_{ij}-g_{j}^{(t)}),\ \forall\  i = 1,\dots,n\ ,
$$
$$
f^{(t+1)}_{i}\leftarrow -\varepsilon \log\left(\sum_{j=1}^{m}\exp\left(\frac{-\left(C_{ij}-g_{j}^{(t)}-m_{i}(g)\right)}{\varepsilon}\right)\beta_{j}\right)+m_{i}(g),\ \forall\  i=1,\dots,n\ ,
$$
$$
m_{j}(f)\leftarrow \min_{i}(C_{ij}-f_{i}^{(t+1)}),\ \forall\   j=1,\dots,m\
 ,
$$
$$
g^{(t+1)}_{j}\leftarrow -\varepsilon \log\left(\sum_{i=1}^{n}\exp\left(\frac{-\left(C_{ij}-f_{i}^{(t+1)}-m_{j}(f)\right)}{\varepsilon}\right)\alpha_{i}\right)+m_{j}(f),\ \forall\  j=1,\dots,m\ ,
$$
where 
$K=e^{-C/\varepsilon} \in M_{n \times m}(\mathbb{R}),\ $ $\varepsilon >0,\ $ $\alpha \in \mathbb{R}^{n},\ $ $\beta \in \mathbb{R}^{m},\ $
   $f \in \mathbb{R}^{n},\ $ $g \in \mathbb{R}^{m}\ and \ (f^{(0)},g^{(0)})=(f,g)$.



In [ ]:
"""To compute distance matrix"""
def distmat( x, y ):
    return np.sum( x**2, 0 )[:,None] + np.sum( y**2, 0 )[None,:] - 2 * x.transpose().dot( y )

"""To Normalise a vector"""
normalize = lambda a: a/np.sum( a )

"""To Compute P"""
def GetP( u, K, v ):
    return u[:,None] * K * v[None,:]

def plotp( x, col, plt, scale = 200, edgecolors = "k" ):
  return plt.scatter( x[0,:], x[1,:], s = scale, edgecolors = edgecolors,  c = col, cmap = 'plasma', linewidths = 2 )

In [ ]:
def generate_data(N):
    """
     N is a list of the size of the data on x and y
    """
    x = np.random.rand( 2, N[0] ) - 0.5
    theta = 2 * np.pi * np.random.rand( 1, N[1] )
    r = 0.8 + .2 * np.random.rand( 1, N[1] )
    y = np.vstack( ( r * np.cos( theta ), r * np.sin( theta ) ) )
    return x, y

In [ ]:
epsilons = [ 0.5, 0.1, 0.05, 0.01 ]
N = [ 500, 600 ]

### I. Log-domain Sinkhorn using numpy

In [ ]:
# Log domain Sinkhorn
print( "Log domain Sinkhorn using numpu... " )
print( "Doing for (",N[0],N[1],")." )
x, y = generate_data( N )
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
results_logSinkhorn_numpy = []
times_logSinkhorn_numpy   = []
logsinkhornP_numpy        = []
#Cost matrix
C = distmat( x, y )
for eps in epsilons:
  print( "For epsilon = "+str(eps)+":" )    
  print( " |- Iterating" )
  start = time.time()
  logsinkhorn = computational_OT.log_domainSinkhorn_np(   a,
                                                          b,
                                                          C,
                                                          eps )
  out = logsinkhorn.update( max_iterations = 500 )
  results_logSinkhorn_numpy.append( out )
  end = time.time()
  times_logSinkhorn_numpy.append( end - start )
  print( " |- Computing P" )
  print( "" )
  u_opt = np.exp( out['potential_f']/eps )
  K = np.exp( - C/eps )
  v_opt =  np.exp( out['potential_g']/eps )
  P_opt = GetP( u_opt, K, v_opt )
  logsinkhornP_numpy.append( P_opt )
# end for

#### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1),
plt.title( "$||P1 -a||_1+||P1 -b||_1$" )
for i in range( len( results_logSinkhorn_numpy) ):
  error = np.asarray( results_logSinkhorn_numpy[i]['error'] )
  plt.plot( error, label = 'Log-sinkhorn for $\epsilon = $'+ str(epsilons[i]) , linewidth = 2 )
# end for
plt.yscale( 'log' )
plt.legend( loc = "upper right" )
plt.xlabel( "Iterations" )
plt.ylabel( "Error in log-scale" )
plt.savefig( "../Images/Log_domain_Sinkhorn_varyingframework_images/Convergencelog_Sinkhorn_np.pdf", format = 'pdf'  )
plt.show()

### II. Log-domain Sinkhorn using JAX

In [ ]:
# Log domain Sinkhorn using JAX
print("Log domain Sinkhorn using JAX... ")
print( "Doing for (",N[0], N[1],")." )
x, y = generate_data( N )
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
results_logSinkhorn_jax = []
times_logSinkhorn_jax   = []
logsinkhornP_jax     = []      
#Cost matrix
C = distmat( x, y )  
for eps in epsilons:
  print( "For epsilon = "+str(eps)+":" )    
  print( " |- Iterating" )
  start = time.time()
  out = computational_OT.Logdomain_Sinkhorn_jax.update(   a,
                                                          b,
                                                          C,
                                                          eps)
  results_logSinkhorn_jax.append( out )
  end = time.time() 
  times_logSinkhorn_jax.append( end - start )
  print( " |- Computing P" )
  print( "" )
  u_opt = np.exp( out['potential_f']/eps )
  K = np.exp( - C/eps )
  v_opt =  np.exp( out['potential_g']/eps )
  P_opt = GetP( u_opt, K, v_opt )
  logsinkhornP_jax.append( P_opt )
# end for

#### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1),
plt.title( "$||P1 -a||_1+||P1 -b||_1$" )
for i in range( len( results_logSinkhorn_jax ) ):
  error = jnp.asarray( results_logSinkhorn_jax[i]['error'] )
  plt.plot( error, label = 'Log-sinkhorn for $\epsilon = $'+ str(epsilons[i]) , linewidth = 2 )
# end for
plt.yscale( 'log' )
plt.legend( loc = "upper right" )
plt.xlabel( "Iterations" )
plt.ylabel( "Error in log-scale" ) 
plt.savefig( "../Images/Log_domain_Sinkhorn_varyingframework_images/Convergencelog_Sinkhorn_jax.pdf", format = 'pdf'  )
plt.show()

### III. Log-domain Sinkhorn using torch

In [ ]:
def torchdistmat( x, y ):
   return torch.sum( x**2, 0 )[:,None] + torch.sum( y**2 ,0  )[None,:] - 2 * torch.matmul( x.t(), y )
torchnormalize = lambda a: a/torch.sum( a )

In [ ]:
# Log domain Sinkhorn using torch
print("Log domain Sinkhorn using torch... ")
print( "Doing for (",N[0], N[1],")." )
x, y = generate_data( N )
x, y = torch.from_numpy( x ), torch.from_numpy( y )
a = torchnormalize( torch.ones( N[0] ) )
b = torchnormalize( torch.ones( N[1] ) )
results_logSinkhorn_torch = []
times_logSinkhorn_torch   = []
logsinkhornP_torch        = []
#Cost matrix
C = torchdistmat( x, y )
for eps in epsilons:
  print( "For epsilon = "+str(eps)+":" )    
  print( " |- Iterating" )
  start = time.time()
  optimizer = computational_OT.logdomain_Sinkhorn_torch(  a,
                                                          b,
                                                          C,
                                                          eps )
  out = optimizer.update( max_iterations = 500 )
  results_logSinkhorn_torch.append( out )
  end = time.time()
  times_logSinkhorn_torch.append( end - start )
  print( " |- Computing P" )
  print( "" )
  u_opt = np.exp( out['potential_f']/eps )
  K = np.exp( - C/eps )
  v_opt =  np.exp( out['potential_g']/eps )
  P_opt = GetP( u_opt, K, v_opt )
  logsinkhornP_torch.append( P_opt )
# end for

#### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$||P1 -a||_1+||P1 -b||_1$" ),
for i in range(len(results_logSinkhorn_torch)):
    error = results_logSinkhorn_torch[i]['error'] 
    plt.plot( error, label='Log-domain Sinkhorn for $\epsilon = $'+str(epsilons[i]), linewidth = 2  )
# end for
plt.yscale( 'log' )
plt.legend( loc = "upper right" )
plt.savefig( "../Images/Log_domain_Sinkhorn_varyingframework_images/Convergencelog_Sinkhorn_torch.pdf", format = 'pdf'  )
plt.show()

### Time plot: numpy vs jax vs torch

In [ ]:
plt.figure( figsize = ( 10, 4 ) )
plt.plot( list(range(len(epsilons))), times_logSinkhorn_numpy[::-1], label = 'Log-domain Sinkhorn', marker = 'o', linewidth = 2 )
plt.plot( list(range(len(epsilons))), times_logSinkhorn_jax[::-1], label = 'Log-domain Sinkhorn using JAX', marker = 'o', linewidth = 2 )
plt.plot( list(range(len(epsilons))), times_logSinkhorn_torch[::-1], label = 'Log-domain Sinkhorn using torch', marker = 'o', linewidth = 2 )
plt.legend( loc = "upper right" )
plt.xticks( list(range(len(epsilons))), epsilons[::-1] )
plt.xlabel( "$\epsilon$" )
plt.yscale( 'log' )
plt.ylabel( "Time in sec" )
plt.savefig( "../Images/Log_domain_Sinkhorn_varyingframework_images/Timecomparison_log_Sinkhorn_torch.pdf", format = 'pdf'  )
plt.show()